<html>
    <summary></summary>
         <div> <p></p> </div>
         <div style="font-size: 20px; width: 800px;"> 
              <h1>
               <left>Determing Growth Rate</left>
              </h1>
              <p><left>============================================================================</left> </p>
<pre>Course: ASU CBP Summer School 2025
Instructor: Dr. Douglas Shepherd
Contact Info: douglas.shepherd@asu.edu
Authors: Dr. Douglas Shepherd
</pre>
         </div>
    </p>

</html>

<details>
  <summary>Copyright info</summary>

```
Copyright 2018 Griffin Chure

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
```
<details>



<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/QI2lab/2025-CBP-SummerSchool/blob/main/Module5-BacteriaGrowth/M5B_Determining_Growth_Rate.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/QI2lab/2025-CBP-SummerSchool/blob/main/Module5-BacteriaGrowth/M5B_Determining_Growth_Rate.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Learning Objectives
In this lesson, we are going to analyze and fit experimental data of bacteria dividing to determine their growth rate. After completing this lesson, you should be able to:
* Determine the distribution of cell division times from experimental data
* Fit the population growth to the exponential growth equation

# Using microscopy to measure bacterial growth

While we have jsut scraped the surface of growth in class, there is a seemingly endless reservoir of literature that explores the fascinating physiology of cellular growth. For a detailed summary, see [this review](https://jun.ucsd.edu/files/publications/Jun_ROPP_2018.pdf) by Suckjoon Jun and colleagues which gives both a historical accounting of bacterial growth physiology and a view of the future of the field. 
 
As discussed in class and examined numerically in [Module 4A](M4A_ExponentialGrowth.ipynb), the continuous growth phase of a bacterial culture can be stated mathematically using the Exponetial Growth equation,

$$
N(t) = N_0 e^{rt}, \tag{1}
$$

where $N(t)$ is the number of cells at time $t$, $N_0$ is the initial number of cells and $r$ is the growth rate in dimensions of time.   Experimentally, it's difficult to count the number of cells in a large culture, but easy to measure for smaller colonies and cultures.

In this tutorial, we will use a movie of *E. coli* cells growing on a hard agar subtstrate at 100x magnification. Below, we have raw phase microscopy data (left), segmentation of individual cells (center), and the result of tracking individual cells and daughter cells (right).

<p style="text-align:center;">
  <span style="display:inline-block; margin:0 10px; text-align:center;">
    <img src="Figures/data_gif.gif" width="200" alt="Raw E. coli phase data" /><br>
    <strong>Phase data</strong>
  </span>
  <span style="display:inline-block; margin:0 10px; text-align:center;">
    <img src="Figures/mask_gif.gif" width="200" alt="Segemented E. coli phase data" /><br>
    <strong>Segmented</strong>
  </span>
  <span style="display:inline-block; margin:0 10px; text-align:center;">
    <img src="Figures/tracked_gif.gif" width="200" alt="Tracked E. Coli phase data" /><br>
    <strong>Tracked</strong>
  </span>
</p>